# Notebook of the paper "From Pairwise to Higher-order Clustering: A (Hyper-)graph Signal Processing Approach on Brain Functional Connectivity Analysis"

In [1]:
# Auxiliary libraries for the computation and plot of the results
import pathlib
import sys

sys.path.insert(0, "./Background_Scripts")
from glob import glob

from bct import null_model_und_sign
from tqdm import tqdm

from Background_Scripts.HCP_Data_Vis_Schaefer_100Parcels import *
from Background_Scripts.plot_functions import *
from Background_Scripts.ttensor import *
from Background_Scripts.utils import *

# Magic command to load watermark
%load_ext watermark

# Possibility to stop warnings
import warnings

warnings.filterwarnings("ignore")

# Print the versions of used packages in the notebook
%watermark --author "Breno & Fernando" --date --time --python --machine --iversion --watermark


Author: Breno & Fernando

Python implementation: CPython
Python version       : 3.7.16
IPython version      : 7.31.1

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 60 Stepping 3, GenuineIntel
CPU cores   : 8
Architecture: 64bit

numpy     : 1.18.5
pandas    : 1.1.3
networkx  : 2.6.3
sys       : 3.7.16 (default, Jan 17 2023, 16:06:28) [MSC v.1916 64 bit (AMD64)]
matplotlib: 3.3.2
scipy     : 1.5.0
seaborn   : 0.11.0
plotly    : 4.6.0

Watermark: 2.1.0



In [6]:
# Definition of file directories and creation of subdirectories
correlation_matrix_dir = "./Schaefer_100Parcels_Atlas/corr_avr_matrix.txt"
triangle_weights_dir = "./Schaefer_100Parcels_Atlas/hoi_zscored_avr_hyperedges.npy"

brain_hypergraph_models_path = pathlib.Path(
    "./Figures/From_Pairwise_to_Higher-order_Clustering/brain_hypergraph_3D_models/"
)
brain_hypergraph_models_path.mkdir(parents=True, exist_ok=True)

clustering_outcomes_path = pathlib.Path(
    "./Figures/From_Pairwise_to_Higher-order_Clustering/clustering_outcomes/"
)
clustering_outcomes_path.mkdir(parents=True, exist_ok=True)

null_models_path = pathlib.Path(
    "./Figures/From_Pairwise_to_Higher-order_Clustering/null_models/"
)
null_models_path.mkdir(parents=True, exist_ok=True)

binarized_adjacency_matrices_path = pathlib.Path(
    "./Figures/From_Pairwise_to_Higher-order_Clustering/binarized_adjacency_matrices/"
)
binarized_adjacency_matrices_path.mkdir(parents=True, exist_ok=True)

brain_signatures_path = pathlib.Path(
    "./Figures/From_Pairwise_to_Higher-order_Clustering/brain_signatures/"
)
brain_signatures_path.mkdir(parents=True, exist_ok=True)


Load the mean individual adjacency matrix $\mathbf{A}$, and get the edge weights. Furthermore, the HOI weights of the brain hypergraphs, $\mathcal{H}_{II}=\{\mathcal{V},\mathcal{M}\}$ and $\mathcal{H}_{TC}=\{\mathcal{V},\mathcal{T}\}$, are loaded.

In [7]:
A = np.genfromtxt(correlation_matrix_dir)
n_rois = len(A)
m_order = 3
edges = {}
for i in range(n_rois):
    for j in range(i + 1, n_rois):
        edges[(i, j)] = A[i, j]

triangles = list(combinations(range(n_rois), m_order))
hoi = np.load(triangle_weights_dir)
hoi_ii = {}
hoi_tc = {}
for i, triangle in enumerate(triangles):
    hoi_ii[triangle] = hoi[i, 0]
    hoi_tc[triangle] = hoi[i, 1]


Plotting 5% of the strongest wedge weights of the brain graph $\mathcal{G}=\{\mathcal{V}, \mathcal{A}\}$ and 0.3% of the strongest HOI weights (in magnitude) of the brain hypergraphs $\mathcal{H}_{II}=\{\mathcal{V}, \mathcal{M}\}$ and $\mathcal{H}_{TC}=\{\mathcal{V}, \mathcal{T}\}$

In [ ]:
Plot_Brain_Interactions(
    edges,
    0.05,
    node_feature_range=(1, 75),
    movie=f"{brain_hypergraph_models_path}/brain_graph_G.html",
    plot_show=False,
)
Plot_Brain_Interactions(
    hoi_ii,
    0.003,
    node_feature_range=(15, 75),
    movie=f"{brain_hypergraph_models_path}/brain_hypergraph_Hii.html",
    plot_show=False,
)
Plot_Brain_Interactions(
    hoi_tc,
    0.003,
    node_feature_range=(15, 75),
    movie=f"{brain_hypergraph_models_path}/brain_hypergraph_Htc.html",
    plot_show=False,
)


Load hyperedges weighths and generate *ttensor* objects related to the simmetrized adjacency tensors $\mathcal{M}_s$ and $\mathcal{I}_s$

In [9]:
As_ii, As_tc = get_symmetrized_ttensors(triangle_weights_dir, triangles)

Defining $\left(\widehat{\mathbf{M}}_s^{abs}\right)^{(0)}$ and $\left(\widehat{\mathbf{T}}_s^{abs}\right)^{(0)}$, which are the adjacency matrices of $\mathcal{G}_{II}^{(0)}$ and $\mathcal{G}_{TC}^{(0)}$, respectively.

In [10]:
Aii_0 = np.abs(As_ii.fft[:, :, 0])
Atc_0 = np.abs(As_tc.fft[:, :, 0])

Running the spectral clustering algorithm on $\mathbf{A}$, $\left(\widehat{\mathbf{M}}_s^{abs}\right)^{(0)}$ and $\left(\widehat{\mathbf{T}}_s^{abs}\right)^{(0)}$ for $K\in\{2,3,\cdots,30\}$.

In [11]:
k_clusters = range(2, 31)

sc_models_G, silhouette_score_sc_models_G, sc_embeddings_G = graph_spectral_clustering(
    A, k_clusters, shift_operator="laplacian", norm_type="sym", eig_type="U"
)
sc_models_Gii_0, silhouette_score_sc_models_Gii_0, sc_embeddings_Gii_0 = (
    graph_spectral_clustering(
        Aii_0, k_clusters, shift_operator="laplacian", norm_type="sym", eig_type="U"
    )
)
sc_models_Gtc_0, silhouette_score_sc_models_Gtc_0, sc_embeddings_Gtc_0 = (
    graph_spectral_clustering(
        Atc_0, k_clusters, shift_operator="laplacian", norm_type="sym", eig_type="U"
    )
)

Plotting the silhouette scores of the spectral clustering models of $\mathcal{G}$, $\mathcal{G}_{II}^{(0)}$, and $\mathcal{G}_{TC}^{(0)}$

In [ ]:
ylabels = [
    r"Silhouette score $\mathcal{G}$",
    r"Silhouette score $\mathcal{G}_{II}^{(0)}$",
    r"Silhouette score $\mathcal{G}_{TC}^{(0)}$",
]
xlabels = [r"$K_{clusters}$"] * 3

x_ticks = [range(0, k_clusters[-1] + 1, 5)] * 3
y_lims = [[-0.01, 0.8], [-0.01, 0.8], [-0.01, 0.8]]

list_df = [
    pd.DataFrame({"x": k_clusters, "y": silhouette_score_sc_models_G}),
    pd.DataFrame({"x": k_clusters, "y": silhouette_score_sc_models_Gii_0}),
    pd.DataFrame({"x": k_clusters, "y": silhouette_score_sc_models_Gtc_0}),
]

plot_multiple_stem(
    list_df=list_df,
    ncols=3,
    xlabels=xlabels,
    ylabels=ylabels,
    ylims=y_lims,
    xticks=x_ticks,
    figsize=(18, 3),
    fig_name=f"{clustering_outcomes_path}/silhouette_scores_sc_0.pdf",
)

We infer that the silhouette diagrams that effectively fulfill the criteria for good clustering and offer pertinent insights into brain functioning are:
- $K=8$ ($SC=0.367$) for $\mathcal{G}$;
- $K=7$ ($SC=0.351$) for $\mathcal{G}_{II}^{(0)}$;
- $K=8$ ($SC=0.392$) for $\mathcal{G}_{TC}^{(0)}$.
  
The silhouette diagrams can be seen in the following cell.

In [ ]:
plot_models_silhouette_diagrams(
    list_kclusters=[8],
    list_embeddings=sc_embeddings_G,
    list_kmeans_models=sc_models_G,
    nodes_color=schaefer_nodes_color,
    name_dataset="$\mathcal{G}$",
    fig_dir=f"{clustering_outcomes_path}/silhouette_diagrams_sc_G.pdf",
)

plot_models_silhouette_diagrams(
    list_kclusters=[7],
    list_embeddings=sc_embeddings_Gii_0,
    list_kmeans_models=sc_models_Gii_0,
    nodes_color=schaefer_nodes_color,
    name_dataset="$\mathcal{G}_{II}^{(0)}$",
    fig_dir=f"{clustering_outcomes_path}/silhouette_diagrams_sc_Gii_0.pdf",
)

plot_models_silhouette_diagrams(
    list_kclusters=[8],
    list_embeddings=sc_embeddings_Gtc_0,
    list_kmeans_models=sc_models_Gtc_0,
    nodes_color=schaefer_nodes_color,
    name_dataset="$\mathcal{G}_{TC}^{(0)}$",
    fig_dir=f"{clustering_outcomes_path}/silhouette_diagrams_sc_Gtc_0.pdf",
)

In [ ]:
# Plotting pie charts of the selected spectral clustering models and the nodes distribution in each cluster according to the Schaefer's atlas subnets

plot_pie_clusters(
    models_list=sc_models_G,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the spectral clustering model of the brain graph }\mathcal{G}$",
    file_dir=f"{clustering_outcomes_path}/pie_charts_sc_G.pdf",
)

plot_pie_clusters(
    models_list=sc_models_Gii_0,
    k_cluster=7,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the spectral clustering model of the brain hypergraph mode }\mathcal{G}_{II}^{(0)}$",
    file_dir=f"{clustering_outcomes_path}/pie_charts_sc_Gii_0.pdf",
)

plot_pie_clusters(
    models_list=sc_models_Gtc_0,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the spectral clustering model of the brain hypergraph mode }\mathcal{G}_{TC}^{(0)}$",
    file_dir=f"{clustering_outcomes_path}/pie_charts_sc_Gtc_0.pdf",
)

In [ ]:
# Plotting a 3D visualization of the Schaefer's atlas for didactic purposes,
# where each ROI's centroid is accurately positioned within the three-dimensional Euclidean model space.
Plot_Brain_Subnets(
    movie="./Figures/From_Pairwise_to_Higher-order_Clustering/brain_subnets.html",
)

In [ ]:
# Plotting the 3D visualizations of the selected spectral clustering models on the brain.
%run  "./Background_Scripts/HCP_Data_Vis_Schaefer_100Parcels.py"

Plot_Brain_Clusters(
    models_list=sc_models_G,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain graph $\mathcal{G}$",
    movie=f"{clustering_outcomes_path}/brain_clusters_sc_G.html",
)

Plot_Brain_Clusters(
    models_list=sc_models_Gii_0,
    k_cluster=7,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{II}^{(0)}$",
    movie=f"{clustering_outcomes_path}/brain_clusters_sc_Gii_0.html",
)

Plot_Brain_Clusters(
    models_list=sc_models_Gtc_0,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{TC}^{(0)}$",
    movie=f"{clustering_outcomes_path}/brain_clusters_sc_Gtc_0.html",
)

Running the DANMF clustering algorithm on $\mathbf{A}$, $\left(\widehat{\mathbf{M}}_s^{abs}\right)^{(0)}$ and $\left(\widehat{\mathbf{T}}_s^{abs}\right)^{(0)}$ for $K\in\{2,3,\cdots,30\}$.

In [ ]:
k_clusters = range(2, 31)
layers = [58]
pre_iterations = 10
iterations = 5000
lamb = 0.1

g = normalize_matrix(A)
danmf_models_G, silhouette_score_models_danmf_G, danmf_embeddings_G = danmf_clustering(
    g,
    k_clusters,
    layers=layers,
    pre_iterations=pre_iterations,
    iterations=iterations,
    lamb=lamb,
)

g = normalize_matrix(Aii_0)
danmf_models_Gii_0, silhouette_score_models_danmf_Gii_0, danmf_embeddings_Gii_0 = (
    danmf_clustering(
        g,
        k_clusters,
        layers=layers,
        pre_iterations=pre_iterations,
        iterations=iterations,
        lamb=lamb,
    )
)

g = normalize_matrix(Atc_0)
danmf_models_Gtc_0, silhouette_score_models_danmf_Gtc_0, danmf_embeddings_Gtc_0 = (
    danmf_clustering(
        g,
        k_clusters,
        layers=layers,
        pre_iterations=pre_iterations,
        iterations=iterations,
        lamb=lamb,
    )
)

Plotting the silhouette scores of the DANMF clustering models of $\mathcal{G}$, $\mathcal{G}_{II}^{(0)}$, and $\mathcal{G}_{TC}^{(0)}$

In [ ]:
ylabels = [
    r"Silhouette score $\mathcal{G}$",
    r"Silhouette score $\mathcal{G}_{II}^{(0)}$",
    r"Silhouette score $\mathcal{G}_{TC}^{(0)}$",
]
xlabels = [r"$K_{clusters}$"] * 3

x_ticks = [range(0, k_clusters[-1] + 1, 5)] * 3
y_lims = [[-0.01, 1.0], [-0.01, 1.0], [-0.01, 1.0]]

list_df = [
    pd.DataFrame({"x": k_clusters, "y": silhouette_score_models_danmf_G}),
    pd.DataFrame({"x": k_clusters, "y": silhouette_score_models_danmf_Gii_0}),
    pd.DataFrame({"x": k_clusters, "y": silhouette_score_models_danmf_Gtc_0}),
]


plot_multiple_stem(
    list_df=list_df,
    ncols=3,
    xlabels=xlabels,
    ylabels=ylabels,
    ylims=y_lims,
    xticks=x_ticks,
    figsize=(18, 4),
    fig_name=f"{clustering_outcomes_path}/silhouette_scores_danmf_0.pdf",
)

The selected DANMF clustering models are:
- $K=8$ ($SC=0.728$) for $\mathcal{G}$;
- $K=8$ ($SC=0.723$) for $\mathcal{G}_{II}^{(0)}$;
- $K=7$ ($SC=0.727$) for $\mathcal{G}_{TC}^{(0)}$.
  
The silhouette diagrams can be seen in the following cell.

In [ ]:
plot_models_silhouette_diagrams(
    list_kclusters=[8],
    list_embeddings=danmf_embeddings_G,
    list_kmeans_models=danmf_models_G,
    nodes_color=schaefer_nodes_color,
    name_dataset="$\mathcal{G}$",
    fig_dir=f"{clustering_outcomes_path}/silhouette_diagrams_danmf_G.pdf",
)

plot_models_silhouette_diagrams(
    list_kclusters=[8],
    list_embeddings=danmf_embeddings_Gii_0,
    list_kmeans_models=danmf_models_Gii_0,
    nodes_color=schaefer_nodes_color,
    name_dataset="$\mathcal{G}_{II}^{(0)}$",
    fig_dir=f"{clustering_outcomes_path}/silhouette_diagrams_danmf_Gii_0.pdf",
)

plot_models_silhouette_diagrams(
    list_kclusters=[7],
    list_embeddings=danmf_embeddings_Gtc_0,
    list_kmeans_models=danmf_models_Gtc_0,
    nodes_color=schaefer_nodes_color,
    name_dataset="$\mathcal{G}_{TC}^{(0)}$",
    fig_dir=f"{clustering_outcomes_path}/silhouette_diagrams_danmf_Gtc_0.pdf",
)

In [ ]:
# Plotting pie charts of the selected DANMF clustering models and the nodes distribution in each cluster according to the Schaefer's atlas subnets

plot_pie_clusters(
    models_list=danmf_models_G,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the DANMF clustering model of the brain graph }\mathcal{G}$",
    file_dir=f"{clustering_outcomes_path}/pie_charts_danmf_G.pdf",
)

plot_pie_clusters(
    models_list=danmf_models_Gii_0,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the DANMF clustering model of the brain hypergraph mode }\mathcal{G}_{II}^{(0)}$",
    file_dir=f"{clustering_outcomes_path}/pie_charts_danmf_Gii_0.pdf",
)

plot_pie_clusters(
    models_list=danmf_models_Gtc_0,
    k_cluster=7,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the DANMF clustering model of the brain hypergraph mode }\mathcal{G}_{TC}^{(0)}$",
    file_dir=f"{clustering_outcomes_path}/pie_charts_danmf_Gtc_0.pdf",
)

In [ ]:
# Plotting the 3D visualizations of the selected DANMF clustering models on the brain.

Plot_Brain_Clusters(
    models_list=danmf_models_G,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain graph $\mathcal{G}$",
    movie=f"{clustering_outcomes_path}/brain_clusters_danmf_G.html",
)

Plot_Brain_Clusters(
    models_list=danmf_models_Gii_0,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{II}^{(0)}$",
    movie=f"{clustering_outcomes_path}/brain_clusters_danmf_Gii_0.html",
)

Plot_Brain_Clusters(
    models_list=danmf_models_Gtc_0,
    k_cluster=7,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{TC}^{(0)}$",
    movie=f"{clustering_outcomes_path}/brain_clusters_danmf_Gtc_0.html",
)

Analyzing the average of the tubal scalars of $\widehat{\mathcal{M}}_s^{abs}, \widehat{\mathcal{T}}_s^{abs}$:

In [ ]:
plot_mean_tubal_scalars(
    As_ii.fft,
    range(0, 51),
    figsize=(9, 4),
    step_xticks=2,
    title=r"Strength of the brain hypergraph modes $\mathcal{G}_{II}^{(k)}=\{\mathcal{V},\left(\widehat{\mathbf{M}}_s^{abs}\right)^{(k)}\}$",
    fig_dir=f"{brain_signatures_path}/mean_individual_strength_Gii.pdf",
)

plot_mean_tubal_scalars(
    As_tc.fft,
    range(0, 51),
    figsize=(9, 4),
    step_xticks=2,
    title=r"Strength of the brain hypergraph modes $\mathcal{G}_{TC}^{(k)}=\{\mathcal{V},\left(\widehat{\mathbf{T}}_s^{abs}\right)^{(k)}\}$",
    fig_dir=f"{brain_signatures_path}/mean_individual_strength_Gtc.pdf",
)

Defining $\left(\widehat{\mathbf{M}}_s^{abs}\right)^{(4)}$ and $\left(\widehat{\mathbf{T}}_s^{abs}\right)^{(4)}$, which are the adjacency matrices of $\mathcal{G}_{II}^{(4)}$ and $\mathcal{G}_{TC}^{(4)}$, respectively.

In [23]:
Aii_4 = np.abs(As_ii.fft[:, :, 4])
Atc_4 = np.abs(As_tc.fft[:, :, 4])

Running the spectral clustering algorithm on $\left(\widehat{\mathbf{M}}_s^{abs}\right)^{(4)}$ and $\left(\widehat{\mathbf{T}}_s^{abs}\right)^{(4)}$ for $K\in\{2,3,\cdots,30\}$.

In [24]:
k_clusters = range(2, 31)

sc_models_Gii_4, silhouette_score_sc_models_Gii_4, sc_embeddings_Gii_4 = (
    graph_spectral_clustering(
        Aii_4, k_clusters, shift_operator="laplacian", norm_type="sym", eig_type="U"
    )
)
sc_models_Gtc_4, silhouette_score_sc_models_Gtc_4, sc_embeddings_Gtc_4 = (
    graph_spectral_clustering(
        Atc_4, k_clusters, shift_operator="laplacian", norm_type="sym", eig_type="U"
    )
)

The selected spectral clustering models are:
- $K=5$ ($SC=0.432$) for $\mathcal{G}_{II}^{(4)}$;
- $K=6$ ($SC=0.502$) for $\mathcal{G}_{TC}^{(4)}$.
  
The silhouette diagrams can be seen in the following cell.

In [ ]:
plot_models_silhouette_diagrams(
    list_kclusters=[5],
    list_embeddings=sc_embeddings_Gii_4,
    list_kmeans_models=sc_models_Gii_4,
    nodes_color=schaefer_nodes_color,
    name_dataset="$\mathcal{G}_{II}^{(4)}$",
    fig_dir=f"{clustering_outcomes_path}/silhouette_diagrams_sc_Gii_4.pdf",
)

plot_models_silhouette_diagrams(
    list_kclusters=[6],
    list_embeddings=sc_embeddings_Gtc_4,
    list_kmeans_models=sc_models_Gtc_4,
    nodes_color=schaefer_nodes_color,
    name_dataset="$\mathcal{G}_{TC}^{(4)}$",
    fig_dir=f"{clustering_outcomes_path}/silhouette_diagrams_sc_Gtc_4.pdf",
)

In [ ]:
# Plotting pie charts of the selected spectral clustering models and the nodes distribution in each cluster according to the Schaefer's atlas subnets

plot_pie_clusters(
    models_list=sc_models_Gii_4,
    k_cluster=5,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the spectral clustering model of the brain hypergraph mode }\mathcal{G}_{II}^{(4)}$",
    file_dir=f"{clustering_outcomes_path}/pie_charts_sc_Gii_4.pdf",
)

plot_pie_clusters(
    models_list=sc_models_Gtc_4,
    k_cluster=6,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the spectral clustering model of the brain hypergraph mode }\mathcal{G}_{TC}^{(4)}$",
    file_dir=f"{clustering_outcomes_path}/pie_charts_sc_Gtc_4.pdf",
)

In [ ]:
# Plotting the 3D visualizations of the selected spectral clustering models on the brain.

Plot_Brain_Clusters(
    models_list=sc_models_Gii_4,
    k_cluster=5,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{II}^{(4)}$",
    movie=f"{clustering_outcomes_path}/brain_clusters_sc_Gii_4.html",
)

Plot_Brain_Clusters(
    models_list=sc_models_Gtc_4,
    k_cluster=6,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{TC}^{(4)}$",
    movie=f"{clustering_outcomes_path}/brain_clusters_sc_Gtc_4.html",
)

Running the DANMF clustering algorithm on $\left(\widehat{\mathbf{M}}_s^{abs}\right)^{(4)}$ and $\left(\widehat{\mathbf{T}}_s^{abs}\right)^{(4)}$ for $K\in\{2,3,\cdots,30\}$.

In [ ]:
k_clusters = range(2, 31)
layers = [58]
pre_iterations = 10
iterations = 5000
lamb = 0.1

g = normalize_matrix(Aii_4)
danmf_models_Gii_4, silhouette_score_models_danmf_Gii_4, danmf_embeddings_Gii_4 = (
    danmf_clustering(
        g,
        layers=layers,
        k_clusters=k_clusters,
        pre_iterations=pre_iterations,
        iterations=iterations,
        lamb=lamb,
    )
)

g = normalize_matrix(Atc_4)
danmf_models_Gtc_4, silhouette_score_models_danmf_Gtc_4, danmf_embeddings_Gtc_4 = (
    danmf_clustering(
        g,
        layers=layers,
        k_clusters=k_clusters,
        pre_iterations=pre_iterations,
        iterations=iterations,
        lamb=lamb,
    )
)

The selected DANMF clustering models are:
- $K=4$ ($SC=0.668$) for $\mathcal{G}_{II}^{(4)}$;
- $K=5$ ($SC=0.639$) for $\mathcal{G}_{TC}^{(4)}$.
  
The silhouette diagrams can be seen in the following cell.

In [ ]:
plot_models_silhouette_diagrams(
    list_kclusters=[4],
    list_embeddings=danmf_embeddings_Gii_4,
    list_kmeans_models=danmf_models_Gii_4,
    nodes_color=schaefer_nodes_color,
    name_dataset="$\mathcal{G}_{II}^{(4)}$",
    fig_dir=f"{clustering_outcomes_path}/silhouette_diagrams_danmf_Gii_4.pdf",
)

plot_models_silhouette_diagrams(
    list_kclusters=[5],
    list_embeddings=danmf_embeddings_Gtc_4,
    list_kmeans_models=danmf_models_Gtc_4,
    nodes_color=schaefer_nodes_color,
    name_dataset="$\mathcal{G}_{TC}^{(4)}$",
    fig_dir=f"{clustering_outcomes_path}/silhouette_diagrams_danmf_Gtc_4.pdf",
)

In [ ]:
# Plotting pie charts of the selected DANMF clustering models and the nodes distribution in each cluster according to the Schaefer's atlas subnets

plot_pie_clusters(
    models_list=danmf_models_Gii_4,
    k_cluster=4,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the DANMF clustering model of the brain hypergraph mode }\mathcal{G}_{II}^{(4)}$",
    file_dir=f"{clustering_outcomes_path}/pie_charts_danmf_Gii_4.pdf",
)

plot_pie_clusters(
    models_list=danmf_models_Gtc_4,
    k_cluster=5,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the DANMF clustering model of the brain hypergraph mode }\mathcal{G}_{TC}^{(4)}$",
    file_dir=f"{clustering_outcomes_path}/pie_charts_danmf_Gtc_4.pdf",
)

In [ ]:
# Plotting the 3D visualizations of the selected DANMF clustering models on the brain.

Plot_Brain_Clusters(
    models_list=danmf_models_Gii_4,
    k_cluster=4,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{II}^{(4)}$",
    movie=f"{clustering_outcomes_path}/brain_clusters_danmf_Gii_4.html",
)

Plot_Brain_Clusters(
    models_list=danmf_models_Gtc_4,
    k_cluster=5,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{TC}^{(4)}$",
    movie=f"{clustering_outcomes_path}/brain_clusters_danmf_Gtc_4.html",
)

In [ ]:
# Defining a Pandas Dataframe that contains all the selected clustering models and Schaefer's subnet labels.

df_cluster_labels_selected = pd.DataFrame(
    {
        "subnet": schaefer_nodes_subnet,
        "schaefer": schaefer_nodes_label,
        "sc_G": get_clusters_label(sc_models_G, kcluster=8),
        "sc_Gii_0": get_clusters_label(sc_models_Gii_0, kcluster=7),
        "sc_Gtc_0": get_clusters_label(sc_models_Gtc_0, kcluster=8),
        "danmf_G": get_clusters_label(danmf_models_G, kcluster=8),
        "danmf_Gii_0": get_clusters_label(danmf_models_Gii_0, kcluster=8),
        "danmf_Gtc_0": get_clusters_label(danmf_models_Gtc_0, kcluster=7),
        "sc_Gii_4": get_clusters_label(sc_models_Gii_4, kcluster=5),
        "sc_Gtc_4": get_clusters_label(sc_models_Gtc_4, kcluster=6),
        "danmf_Gii_4": get_clusters_label(danmf_models_Gii_4, kcluster=4),
        "danmf_Gtc_4": get_clusters_label(danmf_models_Gtc_4, kcluster=5),
    }
)
print(df_cluster_labels_selected)

The following cell generate the null models of the brain graph $\mathcal{G}$ and brain hypergraph modes $\mathcal{G}^{(0)}_{II}$, $\mathcal{G}^{(0)}_{TC}$, $\mathcal{G}^{(4)}_{II}$ and $\mathcal{G}^{(4)}_{TC}$ by rewiring its connections using the Rubinov and Sporns (2011) algorithm.
Subsequently, it plots the edge weight distribution and node degree distribution of the original networks and its rewired versions, for comparison purposes.

In [ ]:
bin_swaps = 100
wei_freq = 0.5
seed = 42

A_null, _ = null_model_und_sign(A, bin_swaps=bin_swaps, wei_freq=wei_freq, seed=seed)
Aii_0_null, _ = null_model_und_sign(
    Aii_0, bin_swaps=bin_swaps, wei_freq=wei_freq, seed=seed
)
Atc_0_null, _ = null_model_und_sign(
    Atc_0, bin_swaps=bin_swaps, wei_freq=wei_freq, seed=seed
)
Aii_4_null, _ = null_model_und_sign(
    Aii_4, bin_swaps=bin_swaps, wei_freq=wei_freq, seed=seed
)
Atc_4_null, _ = null_model_und_sign(
    Atc_4, bin_swaps=bin_swaps, wei_freq=wei_freq, seed=seed
)

# Example usage
titles = [
    r"$\mathcal{G}$",
    r"the rewired version of $\mathcal{G}$",
    r"$\mathcal{G}_{II}^{(0)}$",
    r"the rewired version of $\mathcal{G}_{II}^{(0)}$",
    r"$\mathcal{G}_{TC}^{(0)}$",
    r"the rewired version of $\mathcal{G}_{TC}^{(0)}$",
    r"$\mathcal{G}_{II}^{(4)}$",
    r"the rewired version of $\mathcal{G}_{II}^{(4)}$",
    r"$\mathcal{G}_{TC}^{(4)}$",
    r"the rewired version of $\mathcal{G}_{TC}^{(4)}$",
]

plot_edge_weights_and_node_degrees_distributions(
    [
        A,
        A_null,
        Aii_0,
        Aii_0_null,
        Atc_0,
        Atc_0_null,
        Aii_4,
        Aii_4_null,
        Atc_4,
        Atc_4_null,
    ],
    titles,
    figsize=(13, 30),
    fig_dir=f"{null_models_path}/edge_weights_and_node_degrees_distributions.pdf",
)


Generating the the clustering models of the rewired versions of brain graph and brain hypergraph modes, using the same parameters from the clustering models of the original networks ...

In [ ]:
sc_models_G_null, silhouette_score_sc_models_G_null, sc_embeddings_G_null = (
    graph_spectral_clustering(
        A_null, k_clusters=[8], shift_operator="laplacian", norm_type="sym", eig_type="U"
    )
)

(
    sc_models_Gii_0_null,
    silhouette_score_sc_models_Gii_0_null,
    sc_embeddings_Gii_0_null,
) = graph_spectral_clustering(
    Aii_0_null,
    k_clusters=[7],
    shift_operator="laplacian",
    norm_type="sym",
    eig_type="U",
)
(
    sc_models_Gtc_0_null,
    silhouette_score_sc_models_Gtc_0_null,
    sc_embeddings_Gtc_0_null,
) = graph_spectral_clustering(
    Atc_0_null,
    k_clusters=[8],
    shift_operator="laplacian",
    norm_type="sym",
    eig_type="U",
)

(
    sc_models_Gii_4_null,
    silhouette_score_sc_models_Gii_4_null,
    sc_embeddings_Gii_4_null,
) = graph_spectral_clustering(
    Aii_4_null,
    k_clusters=[5],
    shift_operator="laplacian",
    norm_type="sym",
    eig_type="U",
)
(
    sc_models_Gtc_4_null,
    silhouette_score_sc_models_Gtc_4_null,
    sc_embeddings_Gtc_4_null,
) = graph_spectral_clustering(
    Atc_4_null,
    k_clusters=[6],
    shift_operator="laplacian",
    norm_type="sym",
    eig_type="U",
)

g = normalize_matrix(A_null)
danmf_models_G_null, silhouette_score_models_danmf_G_null, danmf_embeddings_G_null = (
    danmf_clustering(
        g,
        k_clusters=[8],
        layers=layers,
        pre_iterations=pre_iterations,
        iterations=iterations,
        lamb=lamb,
    )
)

g = normalize_matrix(Aii_0_null)
(
    danmf_models_Gii_0_null,
    silhouette_score_models_danmf_Gii_0_null,
    danmf_embeddings_Gii_0_null,
) = danmf_clustering(
    g,
    k_clusters=[8],
    layers=layers,
    pre_iterations=pre_iterations,
    iterations=iterations,
    lamb=lamb,
)

g = normalize_matrix(Atc_0_null)
(
    danmf_models_Gtc_0_null,
    silhouette_score_models_danmf_Gtc_0_null,
    danmf_embeddings_Gtc_0_null,
) = danmf_clustering(
    g,
    k_clusters=[7],
    layers=layers,
    pre_iterations=pre_iterations,
    iterations=iterations,
    lamb=lamb,
)

g = normalize_matrix(Aii_4_null)
(
    danmf_models_Gii_4_null,
    silhouette_score_models_danmf_Gii_4_null,
    danmf_embeddings_Gii_4_null,
) = danmf_clustering(
    g,
    k_clusters=[4],
    layers=layers,
    pre_iterations=pre_iterations,
    iterations=iterations,
    lamb=lamb,
)

g = normalize_matrix(Atc_4_null)
(
    danmf_models_Gtc_4_null,
    silhouette_score_models_danmf_Gtc_4_null,
    danmf_embeddings_Gtc_4_null,
) = danmf_clustering(
    g,
    k_clusters=[5],
    layers=layers,
    pre_iterations=pre_iterations,
    iterations=iterations,
    lamb=lamb,
)


Plotting and saving the clustering outcomes (silhouette diagrams, pie charts and 3D brain visualization) of the null models...

In [35]:
# Spectral clusterng outcomes of the rewired networks

plot_models_silhouette_diagrams(
    list_kclusters=[8],
    list_embeddings=sc_embeddings_G_null,
    list_kmeans_models=sc_models_G_null,
    nodes_color=schaefer_nodes_color,
    name_dataset=r"$\mathcal{G}$",
    fig_dir=f"{null_models_path}/silhouette_diagrams_sc_G_null.pdf",
    plot_show=False,
)

plot_models_silhouette_diagrams(
    list_kclusters=[7],
    list_embeddings=sc_embeddings_Gii_0_null,
    list_kmeans_models=sc_models_Gii_0_null,
    nodes_color=schaefer_nodes_color,
    name_dataset=r"$\mathcal{G}_{II}^{(0)}$",
    fig_dir=f"{null_models_path}/silhouette_diagrams_sc_Gii_0_null.pdf",
    plot_show=False,
)

plot_models_silhouette_diagrams(
    list_kclusters=[8],
    list_embeddings=sc_embeddings_Gtc_0_null,
    list_kmeans_models=sc_models_Gtc_0_null,
    nodes_color=schaefer_nodes_color,
    name_dataset=r"$\mathcal{G}_{TC}^{(0)}$",
    fig_dir=f"{null_models_path}/silhouette_diagrams_sc_Gtc_0_null.pdf",
    plot_show=False,
)

plot_models_silhouette_diagrams(
    list_kclusters=[5],
    list_embeddings=sc_embeddings_Gii_4_null,
    list_kmeans_models=sc_models_Gii_4_null,
    nodes_color=schaefer_nodes_color,
    name_dataset=r"$\mathcal{G}_{II}^{(4)}$",
    fig_dir=f"{null_models_path}/silhouette_diagrams_sc_Gii_4_null.pdf",
    plot_show=False,
)

plot_models_silhouette_diagrams(
    list_kclusters=[6],
    list_embeddings=sc_embeddings_Gtc_4_null,
    list_kmeans_models=sc_models_Gtc_4_null,
    nodes_color=schaefer_nodes_color,
    name_dataset=r"$\mathcal{G}_{TC}^{(4)}$",
    fig_dir=f"{null_models_path}/silhouette_diagrams_sc_Gtc_4_null.pdf",
    plot_show=False,
)

plot_pie_clusters(
    models_list=sc_models_G_null,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the spectral clustering model of the brain graph }\mathcal{G}$",
    file_dir=f"{null_models_path}/pie_charts_sc_G_null.pdf",
    plot_show=False,
)

plot_pie_clusters(
    models_list=sc_models_Gii_0_null,
    k_cluster=7,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the spectral clustering model of the brain hypergraph mode }\mathcal{G}_{II}^{(0)}$",
    file_dir=f"{null_models_path}/pie_charts_sc_Gii_0_null.pdf",
    plot_show=False,
)

plot_pie_clusters(
    models_list=sc_models_Gtc_0_null,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the spectral clustering model of the brain hypergraph mode }\mathcal{G}_{TC}^{(0)}$",
    file_dir=f"{null_models_path}/pie_charts_sc_Gtc_0_null.pdf",
    plot_show=False,
)

plot_pie_clusters(
    models_list=sc_models_Gii_4_null,
    k_cluster=5,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the spectral clustering model of the brain hypergraph mode }\mathcal{G}_{II}^{(4)}$",
    file_dir=f"{null_models_path}/pie_charts_sc_Gii_4_null.pdf",
    plot_show=False,
)

plot_pie_clusters(
    models_list=sc_models_Gtc_4_null,
    k_cluster=6,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the spectral clustering model of the brain hypergraph mode }\mathcal{G}_{TC}^{(4)}$",
    file_dir=f"{null_models_path}/pie_charts_sc_Gtc_4_null.pdf",
    plot_show=False,
)

Plot_Brain_Clusters(
    models_list=sc_models_G_null,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain graph $\mathcal{G}$",
    movie=f"{null_models_path}/brain_clusters_sc_G_null.html",
    plot_show=False,
)

Plot_Brain_Clusters(
    models_list=sc_models_Gii_0_null,
    k_cluster=7,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{II}^{(0)}$",
    movie=f"{null_models_path}/brain_clusters_sc_Gii_0_null.html",
    plot_show=False,
)

Plot_Brain_Clusters(
    models_list=sc_models_Gtc_0_null,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{TC}^{(0)}$",
    movie=f"{null_models_path}/brain_clusters_sc_Gtc_0_null.html",
    plot_show=False,
)

Plot_Brain_Clusters(
    models_list=sc_models_Gii_4_null,
    k_cluster=5,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{II}^{(4)}$",
    movie=f"{null_models_path}/brain_clusters_sc_Gii_4_null.html",
    plot_show=False,
)

Plot_Brain_Clusters(
    models_list=sc_models_Gtc_4_null,
    k_cluster=6,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{TC}^{(4)}$",
    movie=f"{null_models_path}/brain_clusters_sc_Gtc_4_null.html",
    plot_show=False,
)


# DANMF clustering outcomes of the rewired networks

plot_models_silhouette_diagrams(
    list_kclusters=[8],
    list_embeddings=danmf_embeddings_G_null,
    list_kmeans_models=danmf_models_G_null,
    nodes_color=schaefer_nodes_color,
    name_dataset=r"$\mathcal{G}$",
    fig_dir=f"{null_models_path}/silhouette_diagrams_danmf_G_null.pdf",
    plot_show=False,
)

plot_models_silhouette_diagrams(
    list_kclusters=[8],
    list_embeddings=danmf_embeddings_Gii_0_null,
    list_kmeans_models=danmf_models_Gii_0_null,
    nodes_color=schaefer_nodes_color,
    name_dataset=r"$\mathcal{G}_{II}^{(0)}$",
    fig_dir=f"{null_models_path}/silhouette_diagrams_danmf_Gii_0_null.pdf",
    plot_show=False,
)

plot_models_silhouette_diagrams(
    list_kclusters=[7],
    list_embeddings=danmf_embeddings_Gtc_0_null,
    list_kmeans_models=danmf_models_Gtc_0_null,
    nodes_color=schaefer_nodes_color,
    name_dataset=r"$\mathcal{G}_{TC}^{(0)}$",
    fig_dir=f"{null_models_path}/silhouette_diagrams_danmf_Gtc_0_null.pdf",
    plot_show=False,
)

plot_models_silhouette_diagrams(
    list_kclusters=[4],
    list_embeddings=danmf_embeddings_Gii_4_null,
    list_kmeans_models=danmf_models_Gii_4_null,
    nodes_color=schaefer_nodes_color,
    name_dataset=r"$\mathcal{G}_{II}^{(4)}$",
    fig_dir=f"{null_models_path}/silhouette_diagrams_danmf_Gii_4_null.pdf",
    plot_show=False,
)

plot_models_silhouette_diagrams(
    list_kclusters=[5],
    list_embeddings=danmf_embeddings_Gtc_4_null,
    list_kmeans_models=danmf_models_Gtc_4_null,
    nodes_color=schaefer_nodes_color,
    name_dataset=r"$\mathcal{G}_{TC}^{(4)}$",
    fig_dir=f"{null_models_path}/silhouette_diagrams_danmf_Gtc_4_null.pdf",
    plot_show=False,
)

plot_pie_clusters(
    models_list=danmf_models_G_null,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the DANMF clustering model of the brain graph }\mathcal{G}$",
    file_dir=f"{null_models_path}/pie_charts_danmf_G_null.pdf",
    plot_show=False,
)

plot_pie_clusters(
    models_list=danmf_models_Gii_0_null,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the DANMF clustering model of the brain hypergraph mode }\mathcal{G}_{II}^{(0)}$",
    file_dir=f"{null_models_path}/pie_charts_danmf_Gii_0_null.pdf",
    plot_show=False,
)

plot_pie_clusters(
    models_list=danmf_models_Gtc_0_null,
    k_cluster=7,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the DANMF clustering model of the brain hypergraph mode }\mathcal{G}_{TC}^{(0)}$",
    file_dir=f"{null_models_path}/pie_charts_danmf_Gtc_0_null.pdf",
    plot_show=False,
)

plot_pie_clusters(
    models_list=danmf_models_Gii_4_null,
    k_cluster=4,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the DANMF clustering model of the brain hypergraph mode }\mathcal{G}_{II}^{(4)}$",
    file_dir=f"{null_models_path}/pie_charts_danmf_Gii_4_null.pdf",
    plot_show=False,
)

plot_pie_clusters(
    models_list=danmf_models_Gtc_4_null,
    k_cluster=5,
    nodes_color=schaefer_nodes_color,
    nodes_subnet=schaefer_nodes_subnet,
    fig_title=r"$\text{Nodes distribution of the DANMF clustering model of the brain hypergraph mode }\mathcal{G}_{TC}^{(4)}$",
    file_dir=f"{null_models_path}/pie_charts_danmf_Gtc_4_null.pdf",
    plot_show=False,
)

Plot_Brain_Clusters(
    models_list=danmf_models_G_null,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain graph $\mathcal{G}$",
    movie=f"{null_models_path}/brain_clusters_danmf_G_null.html",
    plot_show=False,
)

Plot_Brain_Clusters(
    models_list=danmf_models_Gii_0_null,
    k_cluster=8,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{II}^{(0)}$",
    movie=f"{null_models_path}/brain_clusters_danmf_Gii_0_null.html",
    plot_show=False,
)

Plot_Brain_Clusters(
    models_list=danmf_models_Gtc_0_null,
    k_cluster=7,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{TC}^{(0)}$",
    movie=f"{null_models_path}/brain_clusters_danmf_Gtc_0_null.html",
    plot_show=False,
)

Plot_Brain_Clusters(
    models_list=danmf_models_Gii_4_null,
    k_cluster=4,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{II}^{(4)}$",
    movie=f"{null_models_path}/brain_clusters_danmf_Gii_4_null.html",
    plot_show=False,
)

Plot_Brain_Clusters(
    models_list=danmf_models_Gtc_4_null,
    k_cluster=5,
    nodes_color=schaefer_nodes_color,
    title=r"Clusters of the brain hypergraph mode $\mathcal{G}_{TC}^{(4)}$",
    movie=f"{null_models_path}/brain_clusters_danmf_Gtc_4_null.html",
    plot_show=False,
)

The following cell generate PDF files where the binarized adjacency matrices of $\mathcal{G}$, $\mathcal{G}^{(0)}_{II}$, $\mathcal{G}^{(0)}_{TC}$, $\mathcal{G}^{(4)}_{II}$ and $\mathcal{G}^{(4)}_{TC}$ and its modularities are plotted with distinct values of density $D\in\{5\%, 10\%, 15\%, 20\%\}$

In [ ]:
list_density = [0.05, 0.10, 0.15, 0.20]
df_modularity_sc = pd.DataFrame()
df_modularity_danmf = pd.DataFrame()

for density in list_density:
    mod_G_sc, mod_G_danmf = generate_binarized_adjacency_matrices(
        matrix=A,
        density=density,
        subnet_labels=df_cluster_labels_selected["subnet"],
        atlas_cluster_labels=df_cluster_labels_selected["schaefer"],
        sc_cluster_labels=df_cluster_labels_selected["sc_G"],
        danmf_cluster_labels=df_cluster_labels_selected["danmf_G"],
        matrix_name=r"$\mathcal{G}$",
        fig_name=f"{binarized_adjacency_matrices_path}/adjacency_matrix_G_density_{int(density*100)}%.pdf",
        plot_show=False,
    )

    mod_Gii_0_sc, mod_Gii_0_danmf = generate_binarized_adjacency_matrices(
        matrix=Aii_0,
        density=density,
        subnet_labels=df_cluster_labels_selected["subnet"],
        atlas_cluster_labels=df_cluster_labels_selected["schaefer"],
        sc_cluster_labels=df_cluster_labels_selected["sc_Gii_0"],
        danmf_cluster_labels=df_cluster_labels_selected["danmf_Gii_0"],
        matrix_name=r"$\mathcal{G}_{II}^{(0)}$",
        fig_name=f"{binarized_adjacency_matrices_path}/adjacency_matrix_Gii_0_density_{int(density*100)}%.pdf",
        plot_show=False,
    )

    mod_Gtc_0_sc, mod_Gtc_0_danmf = generate_binarized_adjacency_matrices(
        matrix=Atc_0,
        density=density,
        subnet_labels=df_cluster_labels_selected["subnet"],
        atlas_cluster_labels=df_cluster_labels_selected["schaefer"],
        sc_cluster_labels=df_cluster_labels_selected["sc_Gtc_0"],
        danmf_cluster_labels=df_cluster_labels_selected["danmf_Gtc_0"],
        matrix_name=r"$\mathcal{G}_{TC}^{(0)}$",
        fig_name=f"{binarized_adjacency_matrices_path}/adjacency_matrix_Gtc_0_density_{int(density*100)}%.pdf",
        plot_show=False,
    )

    mod_Gii_4_sc, mod_Gii_4_danmf = generate_binarized_adjacency_matrices(
        matrix=Aii_4,
        density=density,
        subnet_labels=df_cluster_labels_selected["subnet"],
        atlas_cluster_labels=df_cluster_labels_selected["schaefer"],
        sc_cluster_labels=df_cluster_labels_selected["sc_Gii_4"],
        danmf_cluster_labels=df_cluster_labels_selected["danmf_Gii_4"],
        matrix_name=r"$\mathcal{G}_{II}^{(4)}$",
        fig_name=f"{binarized_adjacency_matrices_path}/adjacency_matrix_Gii_4_density_{int(density*100)}%.pdf",
        plot_show=False,
    )

    mod_Gtc_4_sc, mod_Gtc_4_danmf = generate_binarized_adjacency_matrices(
        matrix=Atc_4,
        density=density,
        subnet_labels=df_cluster_labels_selected["subnet"],
        atlas_cluster_labels=df_cluster_labels_selected["schaefer"],
        sc_cluster_labels=df_cluster_labels_selected["sc_Gtc_4"],
        danmf_cluster_labels=df_cluster_labels_selected["danmf_Gtc_4"],
        matrix_name=r"$\mathcal{G}_{TC}^{(4)}$",
        fig_name=f"{binarized_adjacency_matrices_path}/adjacency_matrix_Gtc_4_density_{int(density*100)}%.pdf",
        plot_show=False,
    )

    df_modularity_sc[f"D_{int(density * 100)}%"] = [
        mod_G_sc,
        mod_Gii_0_sc,
        mod_Gtc_0_sc,
        mod_Gii_4_sc,
        mod_Gtc_4_sc,
    ]
    df_modularity_danmf[f"D_{int(density * 100)}%"] = [
        mod_G_danmf,
        mod_Gii_0_danmf,
        mod_Gtc_0_danmf,
        mod_Gii_4_danmf,
        mod_Gtc_4_danmf,
    ]

df_modularity_sc.rename(
    index={0: "G", 1: "Gii_0", 2: "Gtc_0", 3: "Gii_4", 4: "Gtc_4"}, inplace=True
)
df_modularity_danmf.rename(
    index={0: "G", 1: "Gii_0", 2: "Gtc_0", 3: "Gii_4", 4: "Gtc_4"}, inplace=True
)
print(df_modularity_sc)
print(df_modularity_danmf)

Load the raw values of the volunteers' HOI hyperedge weights,  and compute the peak values at $k=0,4$ of the tubal scalar average of $\left(\widehat{\mathcal{M}_1}_s^{abs}\right)^{[n]}, \left(\widehat{\mathcal{T}_1}_s^{abs}\right)^{[n]}, \left(\widehat{\mathcal{M}_2}_s^{abs}\right)^{[n]}, \left(\widehat{\mathcal{T}_2}_s^{abs}\right)^{[n]}$ for each $n$-individual brain hypergraph.

*This may take a few minutes*

In [ ]:
files_hoi = glob("./Schaefer_100Parcels_Atlas/hoi_files_raw/*.npy")
idx = files_hoi[0].index("_REST")
subject_ids = [int(file[idx - 11 : idx - 5]) for file in files_hoi]
rest_numbers = [int(file[idx + 5]) for file in files_hoi]

peak_ii_0_mean = []
peak_ii_4_mean = []
peak_tc_0_mean = []
peak_tc_4_mean = []
peak_ii_0_std = []
peak_ii_4_std = []
peak_tc_0_std = []
peak_tc_4_std = []
k = [0, 4]

for file in tqdm(files_hoi):
    As_ii, As_tc = get_symmetrized_ttensors(file, triangles)
    peak_values_tubal_scalar_ii = mean_std_tubal_scalars(np.abs(As_ii.fft), k)
    peak_values_tubal_scalar_tc = mean_std_tubal_scalars(np.abs(As_tc.fft), k)
    peak_ii_0_mean.append(
        peak_values_tubal_scalar_ii.loc[
            peak_values_tubal_scalar_tc["k"] == 0, "mean"
        ].values[0]
    )
    peak_ii_4_mean.append(
        peak_values_tubal_scalar_ii.loc[
            peak_values_tubal_scalar_tc["k"] == 4, "mean"
        ].values[0]
    )
    peak_tc_0_mean.append(
        peak_values_tubal_scalar_tc.loc[
            peak_values_tubal_scalar_tc["k"] == 0, "mean"
        ].values[0]
    )
    peak_tc_4_mean.append(
        peak_values_tubal_scalar_tc.loc[
            peak_values_tubal_scalar_tc["k"] == 4, "mean"
        ].values[0]
    )


df_peaks = pd.DataFrame(
    {
        "Subject": subject_ids,
        "REST": rest_numbers,
        "peak_ii_0_mean": peak_ii_0_mean,
        "peak_ii_4_mean": peak_ii_4_mean,
        "peak_tc_0_mean": peak_tc_0_mean,
        "peak_tc_4_mean": peak_tc_4_mean,
    }
)

file_subjects_info = "./hcp_subject_genders.xlsx"
df_subjects_info = pd.read_excel(
    file_subjects_info, usecols=["Subject", "Gender"], engine="openpyxl"
)
df_subjects_info_peaks = pd.merge(
    df_subjects_info, df_peaks, how="inner", on=["Subject"]
)
print(df_subjects_info_peaks.head(10))

Plotting the peak values correlation between the first and second resting-state scans of all individuals at $k=0.4$ for $II$ and $TC$.

In [ ]:
list_metrics = ["peak_ii_0_mean", "peak_tc_0_mean", "peak_ii_4_mean", "peak_tc_4_mean"]
list_xlabels = [
    "Strength of ${\mathcal{G}^{(0)}_{II_{1}}}^{[n]}$ and ${\mathcal{G}^{(0)}_{TC_{1}}}^{[n]}$",
    "Strength of ${\mathcal{G}^{(4)}_{II_{1}}}^{[n]}$ and ${\mathcal{G}^{(4)}_{TC_{1}}}^{[n]}$",
]
list_ylabels = [
    "Strength of ${\mathcal{G}^{(0)}_{II_{2}}}^{[n]}$ and ${\mathcal{G}^{(0)}_{TC_{2}}}^{[n]}$",
    "Strength of ${\mathcal{G}^{(4)}_{II_{2}}}^{[n]}$ and ${\mathcal{G}^{(4)}_{TC_{2}}}^{[n]}$",
]
plot_correlation_scatters(
    df_subjects_info_peaks,
    list_metrics,
    list_xlabels,
    list_ylabels,
    figsize=(15, 4),
    fig_dir=f"{brain_signatures_path}/correlation_strength_values.pdf",
)


Plotting the distribution of the peak values between males and females in each resting-state scan at $k=0,4$ for $II$ and $TC$.

In [ ]:
list_metrics = ["peak_ii_0_mean", "peak_tc_0_mean", "peak_ii_4_mean", "peak_tc_4_mean"]
list_ylabels = [
    r"Strength of ${\mathcal{G}^{(0)}_{II_{1}}}^{[n]}$ and ${\mathcal{G}^{(0)}_{II_{2}}}^{[n]}$",
    r"Strength of ${\mathcal{G}^{(0)}_{TC_{1}}}^{[n]}$ and ${\mathcal{G}^{(0)}_{TC_{2}}}^{[n]}$",
    r"Strength of ${\mathcal{G}^{(4)}_{II_{1}}}^{[n]}$ and ${\mathcal{G}^{(4)}_{II_{2}}}^{[n]}$",
    r"Strength of ${\mathcal{G}^{(4)}_{TC_{1}}}^{[n]}$ and ${\mathcal{G}^{(4)}_{TC_{2}}}^{[n]}$",
]
plot_metrics_distribution(
    df_subjects_info_peaks,
    list_metrics,
    list_ylabels,
    figsize=(15, 11),
    fig_dir=f"{brain_signatures_path}/distribution_strength_values.pdf",
)
